## Scrape KHMR

Driver for scraping data from Kicking Horse Mountain Resort in Golden, BC, Canada. Scrapes data from the URL:

http://kickinghorseresort.com/conditions/advanced-weather-data/

and outputs weather data for weewx processing and display etc

In [77]:
import pandas as pd
import requests
import bs4
import datetime
import time
#import weewx

In [78]:
# scrape this URL
the_url = 'http://kickinghorseresort.com/conditions/advanced-weather-data/'

In [79]:
# parse the page with requests + BeautifulSoup
the_page = requests.get(the_url)
the_soup = bs4.BeautifulSoup(the_page.content, 'html.parser').findAll()
the_split = str(the_soup).split("\r\n")

In [80]:
# figure out what day it is to search just the data we want
today_day = datetime.datetime.today().day
today_month = datetime.datetime.today().month
today_year = datetime.datetime.today().year

In [87]:
# parse each line of raw HTML, looking for the current date in a specific format
the_date = 0
for line in the_split:
    if line.__contains__(f"   {today_month} {today_day}    "):
        # extract the date and parse
        the_data = [item for item in line.split(" ") if item != ""]
        the_year = today_year
        the_month = int(the_data[0])
        the_day = int(the_data[1])
        if the_data[2][:-2] == "": # handle "midnight"
            the_hour = 0
        else:
            the_hour = int(the_data[2][:-2])  
        new_date = datetime.datetime(year=the_year, month=the_month, day=the_day, hour=the_hour)
        new_date = time.mktime(new_date.timetuple())
        
        # see if current entry is the latest available
        if new_date > the_date:
            the_date = new_date
        # stop when latest date is reached
        else:
            break
print(the_data)

['10', '29', '0', '-5.7', '71', '36', '208', '49', '12.5']


In [69]:
new_date

1667026800.0

In [70]:
the_date

1667102400.0

In [16]:
my_lines = []
for line in the_split:
    if line.__contains__(f"   {today_month} {today_day}    "):
        print(line)
        my_lines.append(line)
        if len(my_lines) > 24:
            break


             10 29       0    -5.7     101      36     208      49    12.5
             10 29     100    -5.7     136      35     202      44    12.5
             10 29     200    -5.6     167      31     200      52    12.5
             10 29     300    -5.6     197      30     209      44    12.5
             10 29     400    -5.6     227      30     212      39    12.4
             10 29     500    -5.7     255      28     214      39    12.4
             10 29     600    -6.0     285      30     209      43    12.4
             10 29     700    -6.3     319      34     206      44    12.4
             10 29     800    -6.3     346      27     212      42    12.3
             10 29     900    -6.5     375      29     207      45    12.5
             10 29    1000    -6.4     410      35     208      51    13.1
             10 29    1100    -5.8     444      34     208      47    16.6
             10 29    1200    -5.3     471      27     207      36    16.6
             10 29    130

In [5]:
my_line = my_lines[6]

In [6]:
my_data = [item for item in my_line.split(" ") if item != ""]

In [7]:
my_date = datetime.datetime(year=today_year, month=int(my_data[0]), day=int(my_data[1]), hour=int(my_data[2][:-2]))

In [11]:
packet = {"dateTime":time.mktime(my_date.timetuple()),
          "usUnits":weewx.METRIC,
          "outTemp":my_data[3],
          "windSpeed":my_data[5],
          "windDir":my_data[6],
          "windGust":my_data[7],
          "supplyVoltage":my_data[8],    
         }

NameError: name 'weewx' is not defined

In [13]:
packet

{'dateTime': 1667048400.0,
 'outTemp': '-6.0',
 'windSpeed': '30',
 'windDir': '209',
 'windGust': '43',
 'supplyVoltage': '12.4'}